In [1]:
%matplotlib inline
import csv, requests, os
import pandas as pd
import numpy as np

## Get data from Google sheets

In [2]:
def make_regular_gsheet_url(doc_id, sheet_id):
    return f"https://docs.google.com/spreadsheets/d/{doc_id}/edit#gid={sheet_id}"

def make_csv_gsheet_url(doc_id, sheet_id):
    return f"https://docs.google.com/spreadsheets/d/{doc_id}/export?format=csv&id={doc_id}&gid={sheet_id}"


GOOGLE_SHEET_ID = '1bvRKCfu9iGllHsOolDjMtbGA_2COddQFoZ7I45Lyn6o'
print("Querying Doc:", make_regular_gsheet_url(GOOGLE_SHEET_ID, "0"))
response = requests.get(make_csv_gsheet_url(GOOGLE_SHEET_ID, "0"))
reader = csv.reader(response.text.splitlines())
header = next(reader)
df = pd.DataFrame(list(reader), columns=header, dtype=int)

# Remove rows when N/A is a filename
df = df[df['Filename'] != 'N/A']
df['filepath'] = 'speeches/' + df.Filename
df['file_exists'] = df['filepath'].apply(lambda x: os.path.isfile(x))
df.head()

Querying Doc: https://docs.google.com/spreadsheets/d/1bvRKCfu9iGllHsOolDjMtbGA_2COddQFoZ7I45Lyn6o/edit#gid=0


/var/folders/fb/4mqgs_t92c99xgv1z3ql1q6w0000gp/T/ipykernel_18871/3888547131.py:13: FutureWarning: Could not cast to int64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(list(reader), columns=header, dtype=int)
/var/folders/fb/4mqgs_t92c99xgv1z3ql1q6w0000gp/T/ipykernel_18871/3888547131.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['filepath'] = 'speeches/' + df.Filename
/var/folders/fb/4mqgs_t92c99xgv1z3ql1q6w0000gp/T/ipykernel_18871/3888547131.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,Filename,State,Governor,Gender,Party,Type of Speech,New Gov?,2020 Contender?,Region,Trifecta Status,Trifecta,Best Transcript URL,Selector,Note,Lesser Transcript URL,New Best Transcript URL,filepath,file_exists
0,Alabama_Inaugural.txt,Alabama,Kay Ivey,Female,R,Inaugural,No,No,South,R trifecta,Trifecta,https://governor.alabama.gov/remarks-speeches/...,,,https://www.al.com/news/2019/01/the-full-text-...,,speeches/Alabama_Inaugural.txt,True
1,Alabama_SOTS.txt,Alabama,Kay Ivey,Female,R,State of the state,No,No,South,R trifecta,Trifecta,https://governor.alabama.gov/remarks-speeches/...,,,,https://governor.alabama.gov/remarks-speeches/...,speeches/Alabama_SOTS.txt,True
3,Alaska_SOTS.txt,Alaska,Mike Dunleavy,Male,R,State of the state,Yes,No,West,Divided government,Divided,https://gov.alaska.gov/newsroom/2019/01/22/201...,,,https://www.adn.com/politics/2019/01/23/watch-...,https://gov.alaska.gov/newsroom/2019/01/22/201...,speeches/Alaska_SOTS.txt,True
4,Arizona_Inaugural.txt,Arizona,Doug Ducey,Male,R,Inaugural,No,No,West,R trifecta,Trifecta,https://azgovernor.gov/governor/news/2019/01/g...,,,,,speeches/Arizona_Inaugural.txt,True
5,Arizona_SOTS.txt,Arizona,Doug Ducey,Male,R,State of the state,No,No,West,R trifecta,Trifecta,https://azgovernor.gov/governor/news/2019/01/g...,,,,https://azgovernor.gov/governor/news/2019/01/g...,speeches/Arizona_SOTS.txt,True


## Filter Data

In [3]:
df = df[df['Type of Speech'].isin(['State of the state','Both'])]
f"Dataset is {len(df)} speeches"

'Dataset is 50 speeches'

## Read Speeches

In [4]:
def get_speeches(df):
    speeches = []
    for path in df['filepath']:
        with open(path) as f:
            text = f.read()
            speeches.append(text)
    return speeches

speeches = get_speeches(df)

## Turn Speeches Into Vectors!

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

## YOU CAN EDIT THESE
y_columns = ['Party', 'Trifecta']
BINARY=True
NGRAMS=1
MIN_DF=0

def ngram_vectorizer(n, binary, min_df):
    return TfidfVectorizer(
        stop_words='english', # 'english' if not custom list
        ngram_range=(1,n),
        binary=binary,
        min_df=min_df
    )


In [6]:
vectorizer = ngram_vectorizer(NGRAMS, binary=BINARY, min_df=MIN_DF)
X = vectorizer.fit_transform(speeches)
y = np.array(df['Party'])

NameError: name 'TfidfVectorizer' is not defined

In [ ]:
# showing you the array!
print(X.shape)
X.toarray()

In [ ]:
# turning it back into a dataframe for visibility
word_vectors = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df[['State', 'Governor', 'Party', 'Trifecta Status']].reset_index().\
    merge(word_vectors, left_index=True, right_index=True)\
    .drop(columns=['index_x', 'index_y']).head()

In [ ]:
print(y)
y = (y == 'R').astype('int')
y

## Train a Classifier

In [ ]:
clf = MultinomialNB(alpha=1.0e-10, class_prior=None, fit_prior=True)
clf.fit(X, y)

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate

In [ ]:
# 5-fold cross-validation
scoring = ['accuracy', 'precision', 'recall', 'f1']
scores = cross_validate(clf, X, y, scoring=scoring, cv=4)
pd.DataFrame(scores).round(2)

In [ ]:
cross_val_score(clf, X, y)

## Make a prediction

In [ ]:
print(speeches[0])

In [ ]:
speech_vector = vectorizer.transform([speeches[0]])

In [ ]:
speech_vector

In [ ]:
clf.predict(speech_vector)

In [ ]:
clf.predict_proba(speech_vector)

## Peek inside the model (coeffeicients on each word)

https://fivethirtyeight.com/features/what-americas-governors-are-talking-about/

see "state-of-the-states.ipynb"

More on this to come as we learn about bayesian classifiers!